# 🎓 School Portal (Streamlit in Jupyter)
This notebook reproduces the `app.py` functionality in a Jupyter/Colab environment using Streamlit.

In [ ]:
!pip install streamlit pandas scikit-learn --quiet

In [ ]:
%%writefile school_portal.py

import streamlit as st
import pandas as pd
import os
import hashlib
import numpy as np
from sklearn.linear_model import LinearRegression

# Page config
st.set_page_config(page_title='School System', layout='wide')

# Files
USERS_FILE = 'users.csv'
STUDENTS_FILE = 'students.csv'
MARKS_FILE = 'marks.csv'
ATTENDANCE_FILE = 'attendance.csv'
RESULTS_FILE = 'term_results.csv'

# Subjects
COMPULSORY = ['English', 'Mathematics', 'Kiswahili', 'Chemistry', 'Biology']
GROUP_1 = ['CRE', 'Physics']
GROUP_2 = ['History', 'Geography']
GROUP_3 = ['Business', 'Agriculture', 'French', 'HomeScience', 'Computer']
ALL_SUBJECTS = COMPULSORY + GROUP_1 + GROUP_2 + GROUP_3

# Password hashing
def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

# Create files if missing
def create_file(file, columns, data=None):
    if not os.path.exists(file):
        pd.DataFrame(data if data else [], columns=columns).to_csv(file, index=False)

create_file(USERS_FILE, ['username', 'password', 'role', 'subject'], [['admin', hash_password('12345'), 'admin', '']])
create_file(STUDENTS_FILE, ['student_name'])
create_file(MARKS_FILE, ['student', 'subject', 'marks'])
create_file(ATTENDANCE_FILE, ['student', 'attendance_percent'])
create_file(RESULTS_FILE, ['student', 'total_marks', 'average', 'grade'])

# Load/save
def load_all():
    return (
        pd.read_csv(USERS_FILE),
        pd.read_csv(STUDENTS_FILE),
        pd.read_csv(MARKS_FILE),
        pd.read_csv(ATTENDANCE_FILE),
        pd.read_csv(RESULTS_FILE),
    )

def save(df, file):
    df.to_csv(file, index=False)

# Grade calculation
def calculate_grade(avg):
    if avg >= 80: return 'A'
    elif avg >= 70: return 'B'
    elif avg >= 60: return 'C'
    elif avg >= 50: return 'D'
    else: return 'E'

# Load data
users, students, marks, attendance, results = load_all()

# Login system
if 'user' not in st.session_state:
    st.title('🎓 School Portal Login')
    username = st.text_input('Username')
    password = st.text_input('Password', type='password')
    if st.button('Login'):
        hashed = hash_password(password)
        match = users[(users.username==username)&(users.password==hashed)]
        if not match.empty:
            st.session_state.user = match.iloc[0].to_dict()
            st.experimental_rerun()
        else:
            st.error('Wrong username or password')
    st.stop()

user = st.session_state.user
role = user['role']
st.sidebar.success(f"Logged in as {user['username']} ({role})")
if st.sidebar.button('Logout'):
    del st.session_state.user
    st.experimental_rerun()

users, students, marks, attendance, results = load_all()

# ==================== ADMIN PANEL ====================
if role=='admin':
    st.header('🛠 Admin Dashboard')
    st.subheader('📋 User Management')
    col1, col2 = st.columns(2)
    with col1:
        new_user = st.text_input('New Username')
        new_password = st.text_input('Password', type='password')
        role_select = st.selectbox('Role', ['teacher','student'])
        subject = st.text_input('Subject (for teacher)') if role_select=='teacher' else ''
        if st.button('Add User'):
            if new_user.strip() and new_password.strip() and new_user not in users.username.values:
                users = pd.concat([users, pd.DataFrame([{'username':new_user,'password':hash_password(new_password),'role':role_select,'subject':subject}])], ignore_index=True)
                save(users, USERS_FILE)
                st.success(f'User {new_user} added')
                st.experimental_rerun()
    with col2:
        del_user = st.selectbox('Select User to Remove', users[users.username!='admin'].username)
        if st.button('Remove User'):
            users = users[users.username!=del_user]
            save(users, USERS_FILE)
            st.success(f'User {del_user} removed')
            st.experimental_rerun()
    st.info(f'Total users (excluding admin): {len(users)-1}')

    st.subheader('👩‍🎓 Student Management')
    col3, col4 = st.columns(2)
    with col3:
        new_student = st.text_input('New Student Name')
        if st.button('Add Student'):
            if new_student.strip() and new_student not in students.student_name.values:
                students = pd.concat([students, pd.DataFrame([{'student_name': new_student}])], ignore_index=True)
                save(students, STUDENTS_FILE)
                st.success(f'Student {new_student} added')
                st.experimental_rerun()
    with col4:
        del_student = st.selectbox('Select Student to Remove', students.student_name)
        if st.button('Remove Student'):
            students = students[students.student_name!=del_student]
            marks = marks[marks.student!=del_student]
            results = results[results.student!=del_student]
            save(students, STUDENTS_FILE)
            save(marks, MARKS_FILE)
            save(results, RESULTS_FILE)
            st.success(f'Student {del_student} removed')
            st.experimental_rerun()
    st.info(f'Total students: {len(students)}')

# ==================== TEACHER PANEL ====================
elif role=='teacher':
    st.header('👩‍🏫 Teacher Dashboard')
    if students.empty:
        st.warning('No students available')
    else:
        tab1, tab2 = st.tabs(['Manual Entry','Upload CSV'])
        # Manual Entry
        with tab1:
            selected_student = st.selectbox('Select Student', students.student_name)
            student_data = []
            for subj in COMPULSORY:
                mark = st.number_input(f'{subj} Marks', min_value=0, max_value=100)
                student_data.append((subj, mark))
            g1 = st.radio('Group 1', GROUP_1)
            g1_mark = st.number_input(f'{g1} Marks', min_value=0, max_value=100)
            student_data.append((g1,g1_mark))
            g2 = st.radio('Group 2', GROUP_2)
            g2_mark = st.number_input(f'{g2} Marks', min_value=0, max_value=100)
            student_data.append((g2,g2_mark))
            g3 = st.radio('Group 3', GROUP_3)
            g3_mark = st.number_input(f'{g3} Marks', min_value=0, max_value=100)
            student_data.append((g3,g3_mark))
            if st.button('Save Marks'):
                marks = marks[marks.student!=selected_student].copy()
                new_marks = pd.DataFrame(student_data, columns=['subject','marks'])
                new_marks.insert(0,'student',selected_student)
                marks = pd.concat([marks,new_marks], ignore_index=True)
                save(marks, MARKS_FILE)
                # results
                total = marks[marks.student==selected_student].marks.sum()
                avg = marks[marks.student==selected_student].marks.mean()
                grade = calculate_grade(avg)
                results = results[results.student!=selected_student].copy()
                results = pd.concat([results, pd.DataFrame([{'student':selected_student,'total_marks':total,'average':round(avg,2),'grade':grade}])], ignore_index=True)
                save(results, RESULTS_FILE)
                st.success('Marks saved')

# ==================== STUDENT PANEL ====================
elif role=='student':
    st.header(f'📊 Results for {user["username"]}')
    student_name = user['username']
    student_marks = marks[marks.student==student_name]
    student_result = results[results.student==student_name]
    if not student_marks.empty:
        st.subheader('Subject Marks')
        st.table(student_marks)
        if len(student_marks)>1:
            st.bar_chart(student_marks.set_index('subject')['marks'])
    if not student_result.empty:
        st.metric('Total Marks', student_result.iloc[0]['total_marks'])
        st.metric('Average', student_result.iloc[0]['average'])
        st.metric('Grade', student_result.iloc[0]['grade'])
    if len(student_marks)>1:
        X = np.arange(len(student_marks)).reshape(-1,1)
        y = student_marks.marks.values
        model = LinearRegression()
        model.fit(X,y)
        pred = model.predict([[len(y)]])[0]
        st.info(f'📈 Predicted next mark: {round(pred,1)}')

## 🚀 Run the Streamlit App
After running the notebook, use the following command in terminal or Jupyter cell:

In [ ]:
!streamlit run school_portal.py